In [3]:
from vedo import *
import os
from ipyvtklink.viewer import ViewInteractiveWidget
import pykitti
import numpy as np
import tensorflow as tf
import time
import pickle

#limit GPU memory ------------------------------------------------
gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)
if gpus:
  try:
    memlim = 4*1024
    tf.config.experimental.set_virtual_device_configuration(gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=memlim)])
  except RuntimeError as e:
    print(e)
#-----------------------------------------------------------------

from tensorflow.math import sin, cos, tan
import tensorflow_probability as tfp
import sys
current = os.getcwd()
parent_directory = os.path.dirname(current)
sys.path.append(parent_directory)
from ICET_spherical import ICET
from utils import R_tf
from metpy.calc import lat_lon_grid_deltas
from pioneer.das.api.platform import Platform
from scipy.spatial.transform import Rotation as R
from pioneer.das.api.egomotion.imu_egomotion_provider import IMUEgomotionProvider as emp 
from matplotlib import pyplot as plt

%load_ext autoreload
%autoreload 2
%autosave 180
%matplotlib notebook

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Autosaving every 180 seconds


# Goal: Estimate acceleration profile $\hat{m}$ Using Newton-Raphson to compensate for distortion in point cloud

Through the first order Taylor approximation we define $\mathbf{h}(\hat{m})$ as the means of voxel centers in cloud $j$ passed through the distortion operator $\mathbb{C}$ according to estimated motion profile $\hat{m}$. Here, $\mathbf{H}$ is the Jacobian of $\mathbf{h}$, which is multiplied by a linear correction factor $\delta m$.

$    y_i = \mathbf{h}(\hat{m}) + \mathbf{H} \delta m $


In [65]:
#load undistorted point cloud
# distorted =  np.load("sample_data/paper_figures/case1/raw_frame_0.npy") # x
distorted =  np.load("sample_data/paper_figures/case3/raw_frame_1.npy") # x, y, & yaw

In [72]:
from remove_motion_basic import linear_correction_old as lc 

m_hat = np.array([0., 0., 0., 0., 0., -1])
new_cloud = lc(distorted, m_hat)
# print(new_cloud)


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [73]:
plt = Plotter(N = 1, axes = 4, bg = (1, 1, 1), interactive = True)
disp=[]
# disp.append(Points(distorted, c = "#CB2314"))
disp.append(Points(new_cloud, c = "#3F5151"))
plt.show(disp, "test viz")
ViewInteractiveWidget(plt.window)

ViewInteractiveWidget(height=1043, layout=Layout(height='auto', width='100%'), width=1280)